In [1]:
import geopandas as gpd
import gtfsutils
import gtfsutils.filter
import gtfsutils.routes
import folium
from util import get_Baltimore_city_map, save_png

stations = gpd.read_file(f'../../raw_data/Baltimore Red Line- Stations.csv')
segments = gpd.read_file(f'../../raw_data/Baltimore Red Line- Alignment.csv')


def get_gtfs(gtfs_path):
    df_dict = gtfsutils.load_gtfs(gtfs_path)
    df_dict['stop_times'].stop_id = df_dict['stop_times'].stop_id.astype(str)
    df_dict['stops'].stop_id = df_dict['stops'].stop_id.astype(str)
    routes = gtfsutils.routes.load_routes_counts(df_dict)
    stops = gtfsutils.load_stops(df_dict)
    routes = routes.to_crs(epsg=3857)
    return stops, routes

def style_function(feature):
    return {
        'fillOpacity': 0.5,
        'weight': 2,  # This controls the thickness
        'color': 'blue'  # This controls the color
    }

In [2]:
gtfs_stops = []
gtfs_routes =[]
for gtfs_path in ['../../processed_data/redline.zip']:
    stops, routes = get_gtfs(gtfs_path)
    gtfs_stops.append(stops)
    gtfs_routes.append(routes)

[UnicodeDecodeError] 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte for .DS_Store


In [3]:
city_map =  get_Baltimore_city_map(segments=segments, zoom_start=13)

for i in range(1):
    for _, row in gtfs_stops[i].iterrows():
        folium.CircleMarker(
            location=[row['geometry'].y, row['geometry'].x],
            radius=1,
            icon=folium.Icon(color='blue')
        ).add_to(city_map)

    folium.GeoJson(gtfs_routes[i], style_function = style_function).add_to(city_map)

for _, station in stations.iterrows():
    folium.Marker(
        location=[station.geometry.y, station.geometry.x],  # Extract latitude and longitude from the geometry
        icon=folium.Icon(color='red'),  # Specify the custom icon,
    ).add_to(city_map)
folium.PolyLine(
    locations=[(lat, lon) for lon, lat in segments['geometry'][0].coords],
    color='red',
    weight=6,
    opacity=1,
    dash_array='5, 10'
).add_to(city_map)

city_map

In [4]:
stations.explore()